# Spider 데이터셋(original english question)에 대한 Llama 3.1 원본 모델 성능 테스트


In [1]:
# Google Drive 연결
from google.colab import drive
drive.mount('/content/drive')

# 경로 설정 (Spider 데이터셋이 저장된 경로로 수정하세요)
SPIDER_DIR = '/content/drive/MyDrive/spider/spider_data'  # 필요에 따라 수정
DATASET_PATH = '/content/drive/MyDrive/spider/spider_data/spider_train_translations_final.csv'  # 필요에 따라 수정

Mounted at /content/drive


In [2]:

!pip uninstall unsloth triton -y
#!pip install triton vllm unsloth tqdm pandas bitsandbytes
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps "xformers<0.0.27" "trl<0.9.0" peft accelerate bitsandbytes


Found existing installation: triton 3.2.0
Uninstalling triton-3.2.0:
  Successfully uninstalled triton-3.2.0
  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-n2uayr3s/unsloth_a6734e5724a54b198f4c7abb5530e3b7
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-n2uayr3s/unsloth_a6734e5724a54b198f4c7abb5530e3b7
  Resolved https://github.com/unslothai/unsloth.git to commit c1b73fa8836aa7e8b9ee13d748369f8f61e1fac5
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 37.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 49.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.4/147.4 kB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 222.8/222.8 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.2/245.2 kB 24.4 MB/s eta 0:00:00
  Attempting uninstall: trl
    Found existing installation: trl 0.18.1
    Uninstalling trl-0.18.1:
      Successfully uninstalled trl-0.18.1


## 모델 및 토크나이저 로드

In [4]:
from unsloth import FastLanguageModel
import torch

# 모델 구성
max_seq_length = 2048  # 필요에 따라 조정
dtype = None  # 자동 감지
load_in_4bit = True
model_name = "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit"

# 모델 및 토크나이저 로드
print("Loading model and tokenizer...")
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_name,
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
)


Unsloth: Patching Xformers to fix some performance issues.
🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


    PyTorch 2.3.0+cu121 with CUDA 1201 (you have 2.6.0+cu124)
    Python  3.11.9 (you have 3.11.13)
  Please reinstall xformers (see https://github.com/facebookresearch/xformers#installing-xformers)
  Memory-efficient attention, SwiGLU, sparse and more won't be available.
  Set XFORMERS_MORE_DETAILS=1 for more details


🦥 Unsloth Zoo will now patch everything to make training faster!
Loading model and tokenizer...
==((====))==  Unsloth 2025.6.1: Fast Llama patching. Transformers: 4.52.4.
   \\   /|    NVIDIA L4. Num GPUs = 1. Max memory: 22.161 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.9. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/55.5k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

Unsloth 2025.6.1 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


# 테스트 수행


In [9]:
## 10. 평가 (Evaluation)

# 필요한 라이브러리 가져오기
import json
import os
import sqlite3
import sys
import random
from tqdm import tqdm



# 평가 도우미 함수
def load_tables_data(tables_file):
    """테이블 스키마 정보를 로드합니다."""
    if not os.path.exists(tables_file):
        print(f"오류: {tables_file}를 찾을 수 없습니다!")
        return None

    print(f"{tables_file}에서 테이블 로딩 중...")
    with open(tables_file, 'r') as f:
        return json.load(f)

def create_schema_mapping(tables_data):
    """db_id에서 스키마 정보로의 매핑 생성."""
    schema_mapping = {}

    for item in tables_data:
        db_id = item["db_id"]
        tables = item["table_names_original"]
        columns = {}
        column_types = {}

        # 열 정보 처리
        for i, (table_id, col_name) in enumerate(item["column_names_original"]):
            if table_id == -1:
                # 특수 "*" 열 건너뛰기
                continue

            table_name = tables[table_id]

            if table_name not in columns:
                columns[table_name] = []
                column_types[table_name] = {}

            # 가능한 경우 열 유형 추가
            col_type = ""
            if "column_types" in item and i < len(item["column_types"]):
                col_type = item["column_types"][i]
                column_types[table_name][col_name] = col_type

            columns[table_name].append(col_name)

        # 외래 키 매핑
        foreign_keys = []
        for fk_pair in item.get("foreign_keys", []):
            if len(fk_pair) == 2:
                col1_idx, col2_idx = fk_pair

                if (col1_idx < len(item["column_names_original"]) and
                    col2_idx < len(item["column_names_original"])):

                    table1_idx, col1_name = item["column_names_original"][col1_idx]
                    table2_idx, col2_name = item["column_names_original"][col2_idx]

                    if table1_idx >= 0 and table2_idx >= 0:
                        table1_name = tables[table1_idx]
                        table2_name = tables[table2_idx]
                        foreign_keys.append({
                            "source": f"{table1_name}.{col1_name}",
                            "target": f"{table2_name}.{col2_name}"
                        })

        # 기본 키 매핑
        primary_keys = []
        for pk in item.get("primary_keys", []):
            if pk < len(item["column_names_original"]):
                table_id, col_name = item["column_names_original"][pk]
                if table_id >= 0:
                    table_name = tables[table_id]
                    primary_keys.append(f"{table_name}.{col_name}")

        schema_mapping[db_id] = {
            "tables": tables,
            "columns": columns,
            "column_types": column_types,
            "foreign_keys": foreign_keys,
            "primary_keys": primary_keys
        }

    return schema_mapping

def get_schema_text(db_id, schema_mapping):
    """주어진 db_id에 대한 스키마 텍스트 생성."""
    if db_id not in schema_mapping:
        return "Schema information not available"

    schema = schema_mapping[db_id]
    tables = schema["tables"]
    columns = schema["columns"]
    column_types = schema["column_types"]
    foreign_keys = schema["foreign_keys"]
    primary_keys = schema["primary_keys"]

    schema_text = "데이터베이스 스키마:\n"

    for table in tables:
        schema_text += f"테이블: {table}\n"
        if table in columns:
            schema_text += "컬럼:\n"
            for column in columns[table]:
                # 기본 키 표시기 추가
                pk_marker = " (기본 키)" if f"{table}.{column}" in primary_keys else ""

                # 가능한 경우 열 유형 추가
                type_info = ""
                if table in column_types and column in column_types[table]:
                    type_info = f" ({column_types[table][column]})"

                schema_text += f"  - {column}{type_info}{pk_marker}\n"

    if foreign_keys:
        schema_text += "\n외래 키 관계:\n"
        for fk in foreign_keys:
            schema_text += f"  - {fk['source']} → {fk['target']}\n"

    return schema_text

def generate_sql_query(model, tokenizer, schema_mapping, question, db_id):
    """주어진 질문과 데이터베이스에 대한 SQL 쿼리를 생성합니다."""
    try:
        # 스키마 텍스트 가져오기
        if db_id in schema_mapping:
            schema_text = get_schema_text(db_id, schema_mapping)
        else:
            schema_text = "Schema information not available"

        # 스키마를 포함한 입력 구성
        input_with_schema = f"당신은 자연어 질문을 SQL 쿼리로 변환하는 전문 AI 어시스턴트입니다. 사용자가 데이터베이스에서 정보를 얻기 위해 일상 언어로 질문하면, 당신은 해당 질문을 정확한 SQL 쿼리로 변환해야 합니다. 서론과 설명은 제외하고, 바로 작업을 수행하세요.\n\n{schema_text}\n\n질문: {question}"

        # 채팅 메시지로 포맷팅
        messages = [{"role": "user", "content": input_with_schema}]

        # 채팅 템플릿 적용
        inputs = tokenizer.apply_chat_template(
            messages,
            tokenize=True,
            add_generation_prompt=True,
            return_tensors="pt"
        ).to(model.device)

        # SQL 쿼리 생성
        outputs = model.generate(
            inputs,
            max_new_tokens=150,
            temperature=0.1,
            do_sample=True,
            use_cache=True
        )

        # 응답 디코딩
        response = tokenizer.decode(outputs[0], skip_special_tokens=True)

        # 생성된 SQL 쿼리 추출
        assistant_part = response.split(input_with_schema)[-1].strip()

        # 응답 정리하여 SQL 쿼리만 추출
        if assistant_part.startswith("assistant"):
            assistant_part = assistant_part[len("assistant"):].strip()

        # SQL과 유사한 첫 번째 문장 찾기
        lines = assistant_part.split("\n")
        sql_query = ""
        in_sql = False
        for line in lines:
            line = line.strip()
            if not line:
                continue

            # SQL 키워드로 쿼리 시작 식별
            if not in_sql and (line.lower().startswith("select") or
                             line.lower().startswith("with") or
                             line.lower().startswith("(select")):
                in_sql = True

            if in_sql:
                sql_query += line + " "

                # 코드 펜스나 종료 표시를 찾으면 중지
                if line.endswith(";") or line.endswith("```"):
                    break

        # 명확한 SQL 쿼리를 찾지 못한 경우 전체 응답 사용
        if not sql_query:
            sql_query = assistant_part

        # 코드 마커, 백틱 등 정리
        sql_query = sql_query.replace("```sql", "").replace("```", "").strip()

        return sql_query

    except Exception as e:
        print(f"SQL 쿼리 생성 오류: {str(e)}")
        return "SELECT * FROM table"  # 오류 발생 시 기본 쿼리

def evaluate_predictions(preds, golds, db_dir, tables_file):
    """
    생성된 SQL 예측을 평가합니다.

    Args:
        preds: [(sql, db_id), ...] 형식의 예측 목록
        golds: [(sql, db_id), ...] 형식의 정답 목록
        db_dir: SQLite 데이터베이스가 있는 디렉토리
        tables_file: tables.json 파일 경로

    Returns:
        정확도 점수를 포함하는 딕셔너리
    """
    # Spider evaluation.py와 유사한 평가 로직 구현
    scores = {
        'exact_match': 0,
        'execution': 0,
        'total': len(preds),
        'details': []  # 각 예제별 세부 평가 결과
    }

    for i, ((pred_sql, db_id), (gold_sql, _)) in enumerate(zip(preds, golds)):
        # 세부 평가 결과 초기화
        result = {
            'index': i,
            'db_id': db_id,
            'pred_sql': pred_sql,
            'gold_sql': gold_sql,
            'exact_match': False,
            'execution_match': False
        }

        # DB 경로 구성
        db_path = os.path.join(db_dir, db_id, f"{db_id}.sqlite")
        if not os.path.exists(db_path):
            print(f"경고: 데이터베이스 파일을 찾을 수 없음: {db_path}")
            result['error'] = f"DB 파일을 찾을 수 없음: {db_path}"
            scores['details'].append(result)
            continue

        # 실행 정확도 평가
        try:
            conn = sqlite3.connect(db_path)
            cursor = conn.cursor()

            # 예측 SQL 실행
            cursor.execute(pred_sql)
            pred_result = cursor.fetchall()
            result['pred_result_count'] = len(pred_result)

            # 정답 SQL 실행
            cursor.execute(gold_sql)
            gold_result = cursor.fetchall()
            result['gold_result_count'] = len(gold_result)

            # 결과 비교
            execution_match = set(pred_result) == set(gold_result)
            if execution_match:
                scores['execution'] += 1
                result['execution_match'] = True

            conn.close()
        except Exception as e:
            result['error'] = f"실행 오류: {str(e)}"

        # 정확한 일치 평가 (문자열 기반 단순 비교)
        if normalize_sql(pred_sql) == normalize_sql(gold_sql):
            scores['exact_match'] += 1
            result['exact_match'] = True

        # 결과 저장
        scores['details'].append(result)

    # 백분율로 변환
    if scores['total'] > 0:
        scores['exact_match_pct'] = scores['exact_match'] / scores['total'] * 100
        scores['execution_pct'] = scores['execution'] / scores['total'] * 100

    return scores

# 평가 실행 함수
def run_evaluation(model_path=None, sample_size=10, show_all=False):
    """
    모델 평가를 실행합니다.

    Args:
        model_path: 평가할 모델의 경로 (None이면 현재 메모리의 모델 사용)
        sample_size: 평가할 예제 수 (None이면 전체 데이터셋 사용)
        show_all: 모든 예제 결과를 표시할지 여부
    """
    try:
        # 필요한 라이브러리 먼저 임포트
        from unsloth import FastLanguageModel
        import torch
        import pandas as pd
        from IPython.display import display

        # 경로 설정 및 존재 확인
        dev_file = os.path.join(SPIDER_DIR, "dev.json")
        tables_file = os.path.join(SPIDER_DIR, "tables.json")
        db_dir = os.path.join(SPIDER_DIR, "database")

        # 경로 확인
        for path, name in [(dev_file, "dev.json"), (tables_file, "tables.json"), (db_dir, "database")]:
            if not os.path.exists(path):
                print(f"{name} 경로를 찾을 수 없습니다: {path}")
                print(f"SPIDER_DIR = {SPIDER_DIR}")
                print("SPIDER_DIR 변수가 올바르게 설정되었는지 확인하세요.")
                return

        # 개발 데이터셋 로드
        print(f"개발 데이터셋 로드 중: {dev_file}")
        with open(dev_file, 'r') as f:
            dev_data = json.load(f)

        # 샘플 크기가 지정된 경우 예제 수 제한
        if sample_size and sample_size < len(dev_data):
            print(f"전체 {len(dev_data)}개 중 {sample_size}개 예제로 샘플링")
            dev_data = random.sample(dev_data, sample_size)

        # 테이블 스키마 로드
        print(f"테이블 스키마 로드 중: {tables_file}")
        tables_data = load_tables_data(tables_file)
        if not tables_data:
            print("테이블 데이터를 로드하지 못했습니다. 평가를 종료합니다.")
            return

        schema_mapping = create_schema_mapping(tables_data)
        print(f"{len(schema_mapping)}개 데이터베이스에 대한 스키마 매핑 생성")

        # 모델이 메모리에 로드된 상태일 때는 그대로 사용, 아니면 로드
        global model, tokenizer
        if model_path:
            print(f"모델을 {model_path}에서 로드합니다...")
            model, tokenizer = FastLanguageModel.from_pretrained(
                model_path,
                device_map="auto",
                load_in_4bit=True
            )

        # 모델 확인
        if 'model' not in globals() or 'tokenizer' not in globals():
            print("모델과 토크나이저가 로드되지 않았습니다.")
            return

        # 추론 모드로 설정
        if hasattr(model, "for_inference"):
            print("모델을 추론 모드로 설정합니다...")
            FastLanguageModel.for_inference(model)

        # SQL 예측 생성
        preds = []
        golds = []
        questions = []

        print("\nSQL 예측 생성 중:")
        for example in tqdm(dev_data):
            question = example["question"]
            db_id = example["db_id"]
            gold_sql = example["query"]

            # SQL 쿼리 생성
            pred_sql = generate_sql_query(model, tokenizer, schema_mapping, question, db_id)

            # 결과 저장
            questions.append(question)
            preds.append((pred_sql, db_id))
            golds.append((gold_sql, db_id))

        # 평가 수행
        print("\n평가 실행 중...")
        scores = evaluate_predictions(preds, golds, db_dir, tables_file)

        # 결과 출력
        print("\n평가 결과:")
        print(f"총 예제 수: {scores['total']}")
        print(f"완전 일치 정확도: {scores['exact_match_pct']:.2f}% ({scores['exact_match']}/{scores['total']})")
        print(f"실행 정확도: {scores['execution_pct']:.2f}% ({scores['execution']}/{scores['total']})")

        # 결과 데이터프레임 생성
        results = []
        for i, detail in enumerate(scores['details']):
            results.append({
                "예제 번호": i + 1,
                "질문": questions[i],
                "DB": detail['db_id'],
                "정확한 일치": "✅" if detail['exact_match'] else "❌",
                "실행 일치": "✅" if detail['execution_match'] else "❌",
                "예측 SQL": detail['pred_sql'],
                "정답 SQL": detail['gold_sql']
            })

        # 데이터프레임 생성
        df = pd.DataFrame(results)

        # show_all이 True이면 모든 예제 표시, 아니면 일부만 표시
        if show_all:
            print(f"\n모든 예제 결과 ({len(results)}개):")
            display(df)
        else:
            # 최대 5개 샘플만 표시
            max_samples = min(5, len(results))
            print(f"\n샘플 예제 결과 ({max_samples}개):")
            display(df.head(max_samples))
            if len(results) > max_samples:
                print(f"...외 {len(results) - max_samples}개 더 있음. 모두 보려면 show_all=True 옵션 사용")

        return df, scores

    except Exception as e:
        print(f"평가 중 오류 발생: {str(e)}")
        import traceback
        traceback.print_exc()
        return None

# 메인 평가 예시 (모든 예제 평가 결과를 보려면 show_all=True 옵션 추가)
# run_evaluation(sample_size=10, show_all=True)  # 10개 예제 전체 결과 표시

In [10]:
# 모든 예제 예측 결과를 보여주는 함수
def show_all_predictions(model_path=None, sample_size=None):
    """
    모든 예제에 대한 SQL 예측 결과를 생성하여 표시합니다.

    Args:
        model_path: 평가할 모델의 경로 (None이면 현재 메모리의 모델 사용)
        sample_size: 평가할 예제 수 (None이면 전체 데이터셋 사용)
    """
    try:
        # 필요한 라이브러리 먼저 임포트
        from unsloth import FastLanguageModel
        import torch
        import pandas as pd
        from IPython.display import display

        # 경로 설정 및 존재 확인
        dev_file = os.path.join(SPIDER_DIR, "dev.json")
        tables_file = os.path.join(SPIDER_DIR, "tables.json")

        # 경로 확인
        for path, name in [(dev_file, "dev.json"), (tables_file, "tables.json")]:
            if not os.path.exists(path):
                print(f"{name} 경로를 찾을 수 없습니다: {path}")
                print(f"SPIDER_DIR = {SPIDER_DIR}")
                return

        # 개발 데이터셋 로드
        print(f"개발 데이터셋 로드 중: {dev_file}")
        with open(dev_file, 'r') as f:
            dev_data = json.load(f)

        # 샘플 크기가 지정된 경우 예제 수 제한
        if sample_size and sample_size < len(dev_data):
            print(f"전체 {len(dev_data)}개 중 {sample_size}개 예제로 샘플링")
            dev_data = random.sample(dev_data, sample_size)

        # 테이블 스키마 로드
        print(f"테이블 스키마 로드 중: {tables_file}")
        tables_data = load_tables_data(tables_file)
        if not tables_data:
            print("테이블 데이터를 로드하지 못했습니다. 종료합니다.")
            return

        schema_mapping = create_schema_mapping(tables_data)
        print(f"{len(schema_mapping)}개 데이터베이스에 대한 스키마 매핑 생성")

        # 모델이 메모리에 로드된 상태일 때는 그대로 사용, 아니면 로드
        global model, tokenizer
        if model_path:
            print(f"모델을 {model_path}에서 로드합니다...")
            model, tokenizer = FastLanguageModel.from_pretrained(
                model_path,
                device_map="auto",
                load_in_4bit=True
            )

        # 모델 확인
        if 'model' not in globals() or 'tokenizer' not in globals():
            print("모델과 토크나이저가 로드되지 않았습니다.")
            return

        # 추론 모드로 설정
        if hasattr(model, "for_inference"):
            print("모델을 추론 모드로 설정합니다...")
            FastLanguageModel.for_inference(model)

        # SQL 예측 생성
        results = []
        print("\nSQL 예측 생성 중:")
        for i, example in enumerate(tqdm(dev_data)):
            question = example["question"]
            db_id = example["db_id"]
            gold_sql = example["query"]

            # SQL 쿼리 생성
            pred_sql = generate_sql_query(model, tokenizer, schema_mapping, question, db_id)

            # 결과 저장
            results.append({
                "예제 #": i+1,
                "질문": question,
                "DB": db_id,
                "예측 SQL": pred_sql,
                "정답 SQL": gold_sql
            })

        # 데이터프레임으로 변환
        df = pd.DataFrame(results)

        # 결과 표시
        print(f"\n총 {len(results)}개 예제 평가 완료")

        # 데이터프레임 표시
        display(df)

        return df

    except Exception as e:
        print(f"오류 발생: {str(e)}")
        import traceback
        traceback.print_exc()
        return None

# 함수 호출 (필요시 주석 해제)
# show_all_predictions(sample_size=10)  # 10개 예제만 표시

In [14]:

# ==============================================================================
# 1. 라이브러리 임포트
# ==============================================================================
import json
import os
import sqlite3
import re
import random
from tqdm import tqdm
import pandas as pd
import torch
from unsloth import FastLanguageModel
from IPython.display import display

# ==============================================================================
# 2. 경로 및 기본 설정
# ==============================================================================
# !!! 중요: 이 경로를 실제 Spider 데이터셋이 있는 디렉토리로 변경하세요 !!!
SPIDER_DIR = '/content/drive/MyDrive/spider/spider_data'  # 예: '/content/spider' 또는 'data/spider'

# 사용할 모델의 Hugging Face 경로
# 예: "unsloth/llama-3.1-8b-Instruct-bnb-4bit"
MODEL_NAME = "unsloth/llama-3.1-8b-Instruct-bnb-4bit"


# ==============================================================================
# 3. 데이터 로딩 및 스키마 처리 헬퍼 함수
# ==============================================================================
def load_tables_data(tables_file):
    """tables.json 파일에서 테이블 스키마 정보를 로드합니다."""
    if not os.path.exists(tables_file):
        print(f"오류: '{tables_file}' 파일을 찾을 수 없습니다!")
        return None
    with open(tables_file, 'r', encoding='utf-8') as f:
        return json.load(f)

def create_schema_mapping(tables_data):
    """db_id를 키로 하는 스키마 정보 딕셔너리를 생성합니다."""
    schema_mapping = {}
    for item in tables_data:
        db_id = item["db_id"]
        tables = item["table_names_original"]
        columns = {tbl: [] for tbl in tables}
        column_types = {tbl: {} for tbl in tables}

        for i, (table_id, col_name) in enumerate(item["column_names_original"]):
            if table_id >= 0:
                table_name = tables[table_id]
                columns[table_name].append(col_name)
                column_types[table_name][col_name] = item["column_types"][i]

        primary_keys = []
        for pk_col_idx in item.get("primary_keys", []):
            table_id, col_name = item["column_names_original"][pk_col_idx]
            if table_id >= 0:
                primary_keys.append(f"{tables[table_id]}.{col_name}")

        schema_mapping[db_id] = {
            "tables": tables,
            "columns": columns,
            "column_types": column_types,
            "primary_keys": primary_keys,
        }
    return schema_mapping

def get_schema_text(db_id, schema_mapping):
    """주어진 db_id에 대한 텍스트 형식의 스키마를 생성합니다."""
    if db_id not in schema_mapping:
        return "Schema information not available"

    schema = schema_mapping[db_id]
    schema_text_parts = []

    for table_name in schema["tables"]:
        schema_text_parts.append(f"테이블: {table_name}")
        cols_info = []
        for col_name in schema["columns"].get(table_name, []):
            col_type = schema["column_types"][table_name][col_name]
            pk_marker = " (기본 키)" if f"{table_name}.{col_name}" in schema["primary_keys"] else ""
            cols_info.append(f"  - {col_name} ({col_type}){pk_marker}")
        schema_text_parts.append("컬럼:\n" + "\n".join(cols_info))

    return "\n".join(schema_text_parts)

# ==============================================================================
# 4. 핵심 로직: SQL 생성 함수
# ==============================================================================
def generate_sql(model, tokenizer, schema_info, question):
    """
    모델에게 Markdown 코드 블록으로 응답하도록 지시하고, 후처리를 통해 SQL만 추출합니다.
    """
    messages = [
        {
            "role": "system",
            "content": "당신은 Text-to-SQL 전문가입니다. 주어진 스키마와 질문에 대한 SQL 쿼리를 생성하세요. 응답은 반드시 ```sql 코드 블록 안에 넣어야 합니다. 다른 어떤 설명도 추가하지 마세요."
        },
        {
            "role": "user",
            "content": f"아래 스키마와 질문을 바탕으로 SQL 쿼리를 markdown 코드 블록으로 작성해줘.\n\n[스키마]:\n{schema_info}\n\n[질문]:\n{question}\n\n[SQL 쿼리]:"
        }
    ]

    inputs = tokenizer.apply_chat_template(
        messages,
        tokenize=True,
        add_generation_prompt=True,
        return_tensors="pt"
    ).to(model.device)

    outputs = model.generate(
        inputs,
        max_new_tokens=150,
        temperature=0.1,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id
    )

    full_response = tokenizer.decode(outputs[0][len(inputs[0]):], skip_special_tokens=True)

    # --- 후처리 로직: ```sql 블록에서 SQL 쿼리 추출 ---
    match = re.search(r'```sql\s*(.*?)\s*```', full_response, re.DOTALL)

    if match:
        sql_query = match.group(1).strip()
        return sql_query
    else:
        # 모델이 코드 블록을 생성하지 않은 경우에 대한 대체(Fallback) 로직
        fallback_match = re.search(r'(?i)(SELECT|WITH|\(SELECT)', full_response)
        if fallback_match:
            return full_response[fallback_match.start():].strip()
        return full_response.strip() # 최악의 경우, 디버깅을 위해 그대로 반환

# ==============================================================================
# 5. 평가 헬퍼 함수
# ==============================================================================
def normalize_sql(sql):
    """SQL 쿼리를 비교하기 쉽도록 정규화합니다."""
    if not isinstance(sql, str):
        return ""
    sql = sql.lower().strip()
    sql = re.sub(r'\s+', ' ', sql)
    sql = re.sub(r'\s*,\s*', ', ', sql)
    sql = re.sub(r'\s*=\s*', ' = ', sql)
    sql = re.sub(r';+\s*$', '', sql)
    return sql

def execute_sql(db_path, sql):
    """SQLite 데이터베이스에서 SQL을 실행하고 결과를 반환합니다."""
    try:
        conn = sqlite3.connect(db_path)
        cursor = conn.cursor()
        cursor.execute(sql)
        results = cursor.fetchall()
        conn.close()
        return True, results
    except Exception as e:
        return False, str(e)

# ==============================================================================
# 6. 메인 평가 실행 함수
# ==============================================================================
def run_evaluation_ko_from_csv(model, tokenizer, n_samples=10, random_seed=42):
    """
    CSV 파일에서 한국어 질문을 사용하여 모델을 평가합니다.
    """
    try:
        csv_path = os.path.join(SPIDER_DIR, 'spider_validation_final.csv')
        tables_file = os.path.join(SPIDER_DIR, 'tables.json')
        db_dir = os.path.join(SPIDER_DIR, 'database')

        # 파일 존재 확인
        for path in [csv_path, tables_file, db_dir]:
            if not os.path.exists(path):
                print(f"❌ 필수 파일/디렉토리를 찾을 수 없습니다: {path}")
                print(f"SPIDER_DIR ('{SPIDER_DIR}') 경로가 올바른지 확인하세요.")
                return

        print(f"✅ 데이터셋 파일 확인 완료.")
        print(f"Loading CSV file from {csv_path}…")
        df = pd.read_csv(csv_path)

        # 스키마 정보 로드
        tables_data = load_tables_data(tables_file)
        schema_dict = create_schema_mapping(tables_data)

        # 랜덤 샘플 선택
        random.seed(random_seed)
        sample_indices = random.sample(range(len(df)), min(n_samples, len(df)))
        sampled_df = df.iloc[sample_indices]

        print(f"\n🚀 총 {len(sampled_df)}개 랜덤 샘플에 대한 평가를 시작합니다…\n" + "="*80)

        results = []

        for i, (idx, row) in enumerate(sampled_df.iterrows()):
            db_id = row['db_id']
            question = row['question']
            gold_sql = row['query']

            print(f"\nExample {i+1}/{len(sampled_df)}:")
            print(f"Database: {db_id}")
            print(f"Question (KO): {question}")
            print(f"Gold SQL: {gold_sql}")

            schema_info = get_schema_text(db_id, schema_dict)
            pred_sql = generate_sql(model, tokenizer, schema_info, question)

            print(f"Generated SQL: {pred_sql}")

            # 정확한 일치(Exact Match) 평가
            exact_match = normalize_sql(gold_sql) == normalize_sql(pred_sql)
            print(f"Exact Match: {'✅' if exact_match else '❌'}")

            # 실행 정확도(Execution Accuracy) 평가
            db_path = os.path.join(db_dir, db_id, f"{db_id}.sqlite")
            execution_match = False
            execution_error = None

            if os.path.exists(db_path):
                gold_success, gold_result = execute_sql(db_path, gold_sql)
                pred_success, pred_result = execute_sql(db_path, pred_sql)

                if not pred_success:
                    execution_error = pred_result
                    print(f"Execution: ❌ (Error: {str(execution_error)[:100]}...)")
                elif not gold_success:
                     print(f"Execution: ⚠️ (Gold SQL 실행 오류)")
                else:
                    execution_match = (set(gold_result) == set(pred_result))
                    print(f"Execution Match: {'✅' if execution_match else '❌'}")
            else:
                execution_error = "DB file not found"
                print(f"Execution: ⚠️ (DB 파일을 찾을 수 없음)")

            print("-" * 80)

            results.append({
                'index': idx, 'db_id': db_id, 'question': question,
                'gold_sql': gold_sql, 'generated_sql': pred_sql,
                'exact_match': exact_match, 'execution_match': execution_match,
                'execution_error': execution_error
            })

        # 최종 결과 요약
        total = len(results)
        em_count = sum(r['exact_match'] for r in results)
        ex_count = sum(r['execution_match'] for r in results)

        print(f"\n{'='*80}\n📊 최종 평가 요약 (총 {total}개 예제):\n{'='*80}")
        print(f"  - 완전 일치 (EM): {em_count} / {total} ({em_count/total:.2%})")
        print(f"  - 실행 정확도 (EX): {ex_count} / {total} ({ex_count/total:.2%})")
        print(f"{'='*80}")

        return pd.DataFrame(results)

    except Exception as e:
        print(f"🚨 평가 중 심각한 오류 발생: {e}")
        import traceback
        traceback.print_exc()
        return None

# ==============================================================================
# 7. 스크립트 실행
# ==============================================================================
if __name__ == '__main__':

    # 평가 실행
    # n_samples: 평가할 샘플 수. 전체(1034개)를 평가하려면 1034로 설정
    evaluation_results_df = run_evaluation_ko_from_csv(
        model,
        tokenizer,
        n_samples=1034, # 테스트를 위해 10개만 실행, 전체는 1034로 변경
        random_seed=15
    )

    # 결과 데이터프레임 출력
    if evaluation_results_df is not None:
        print("\n\n📋 상세 평가 결과 테이블:")
        display(evaluation_results_df)



스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
Question (KO): Find the names of stores whose number products is more than the average number of products.
Gold SQL: SELECT name FROM shop WHERE number_products  >  (SELECT avg(number_products) FROM shop)
Generated SQL: SELECT s.name
FROM shop s
JOIN hiring h ON s.shop_id = h.shop_id
JOIN employee e ON h.employee_id = e.employee_id
WHERE h.number_products > (SELECT AVG(number_products) FROM hiring);
Exact Match: ❌
Execution: ❌ (Error: no such column: h.number_products...)
--------------------------------------------------------------------------------

Example 734/1034:
Database: car_1
Question (KO): What is the average horsepower of the cars before 1980?
Gold SQL: SELECT avg(horsepower) FROM CARS_DATA WHERE YEAR  <  1980;
Generated SQL: SELECT AVG(Horsepower)
FROM cars_data
WHERE Year < 1980;
Exact Match: ✅
Execution Match: ✅
--------------------------------------------------------------------------------

Example 735/1034:
Database: wta_1
Question 

,index,db_id,question,gold_sql,generated_sql,exact_match,execution_match,execution_error
0,427,museum_visit,Find the number of visitors who did not visit ...,SELECT count(*) FROM visitor WHERE id NOT IN (...,SELECT COUNT(*) \nFROM visitor \nWHERE visit_i...,False,False,no such column: visit_id
1,23,concert_singer,"For each stadium, how many concerts play there?","SELECT T2.name , count(*) FROM concert AS T1 ...",SELECT COUNT(*) FROM stadium WHERE name = 'con...,False,False,You can only execute one statement at a time.
2,74,pets_1,What is the average weight for each type of pet?,"SELECT avg(weight) , pettype FROM pets GROUP ...","SELECT p.pet_type, AVG(p.weight) \nFROM pets p...",False,False,no such table: students
3,323,cre_Doc_Template_Mgt,What are the ids of templates with template ty...,SELECT template_id FROM Templates WHERE templa...,"SELECT \n T1.Template_ID,\n T1.Template_Type...",False,False,None
4,489,wta_1,Find the number of players for each hand type.,"SELECT count(*) , hand FROM players GROUP BY ...","SELECT \n hand,\n COUNT(*) \nFROM \n player...",False,False,None
...,...,...,...,...,...,...,...,...
1029,727,world_1,Give the total surface area covered by countri...,SELECT sum(SurfaceArea) FROM country WHERE Con...,"SELECT \n c1.name, \n c2.name, \n c1.surfac...",False,False,no such table: cities
1030,282,employee_hire_evaluation,What are the names of the employees who never ...,SELECT name FROM employee WHERE Employee_ID NO...,SELECT E.name\nFROM employee E\nWHERE E.Employ...,False,False,ambiguous column name: E.Employee_ID
1031,223,flight_2,Which city has most number of departing flights?,SELECT T1.City FROM AIRPORTS AS T1 JOIN FLIGHT...,SELECT T2.City\nFROM airlines T1\nJOIN flights...,False,False,no such column: T2.City
1032,1013,singer,"For each citizenship, how many singers are fro...","SELECT Citizenship , COUNT(*) FROM singer GRO...","SELECT s.citizenship, COUNT(s.singer_id) \nFRO...",False,True,None
